<a href="https://colab.research.google.com/github/neuromatch/NeuroAI_Course/blob/main/tutorials/W2D2_NeuroSymbolicMethods/student/W2D2_Tutorial4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/neuromatch/NeuroAI_Course/main/tutorials/W2D2_NeuroSymbolicMethods/student/W2D2_Tutorial4.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# (BONUS) Tutorial 4: VSA Analogies

**Week 2, Day 2: Neuro-Symbolic Methods**

**By Neuromatch Academy**

__Content creators:__ P. Michael Furlong, Chris Eliasmith

__Content reviewers:__ Hlib Solodzhuk, Patrick Mineault, Aakash Agrawal, Alish Dipani, Hossein Rezaei, Yousef Ghanbari, Mostafa Abdollahi, Alex Murphy

__Production editors:__ Konstantine Tsafatinos, Ella Batty, Spiros Chavlis, Samuele Bolotta, Hlib Solodzhuk, Alex Murphy


___


# Tutorial Objectives

This tutorial will present you with a couple of toy examples using the basic operations of vector symbolic algebras. We will further show how these can generalize to new knowledge. If you are familiar with the basics of semantic algebra on word embeddings, you already understand the basics of what we'll be demonstrating in this tutorial. If not, don't worry, this tutorial is designed to be highly self-contained. If you're interested in learning more about word embedding algebra, then we encourage you to use your search engine of choice and learn more after completing the code exercises given below.

In [ ]:
# @title Tutorial slides
# @markdown These are the slides for the videos in all tutorials today

from IPython.display import IFrame
link_id = "jybuw"

print(f"If you want to download the slides: 'https://osf.io/download/{link_id}'")

IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{link_id}/?direct%26mode=render", width=854, height=480)

---
# Setup (Colab Users: Please Read)

Note that because this tutorial relies on some special Python packages, these packages have requirements for specific versions of common scientific libraries, such as `numpy`. If you're in Google Colab, then as of May 2025, this comes with a later version (2.0.2) pre-installed. We require an older version (we'll be installing `1.24.4`). This causes Colab to force a session restart and then re-running of the installation cells for the new version to take effect. When you run the cell below, you will be prompted to restart the session. This is *entirely expected* and you haven't done anything wrong. Simply click 'Restart' and then run the cells as normal. 

In [ ]:
# @title Install dependencies and import feedback gadget

!pip install numpy==1.24.4 --quiet
!pip install scikit-learn==1.6.1 --quiet
!pip install scipy==1.15.3 --quiet
!pip install git+https://github.com/neuromatch/sspspace@neuromatch --no-deps --quiet
!pip install nengo==4.0.0 --quiet
!pip install nengo_spa==2.0.0 --quiet
!pip install --quiet matplotlib ipywidgets vibecheck
!pip install --quiet numpy matplotlib ipywidgets scipy scikit-learn vibecheck

from vibecheck import DatatopsContentReviewContainer
def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_neuroai",
            "user_key": "wb2cxze8",
        },
    ).render()


feedback_prefix = "W2D2_T4"

Notice that exactly the `neuromatch` branch of `sspspace` should be installed! Otherwise, some of the functionality (like `optimize` parameter in the `DiscreteSPSpace` initialization) won't work.

In [ ]:
# @title Imports

#working with data
import numpy as np

#plotting
import matplotlib.pyplot as plt
import logging

#interactive display
# import ipywidgets as widgets

#modeling
import sspspace
import nengo_spa as spa
from scipy.special import softmax
from sklearn.metrics import log_loss
from sklearn.neural_network import MLPRegressor
from nengo_spa.algebras.hrr_algebra import HrrProperties, HrrAlgebra
from nengo_spa.vector_generation import VectorsWithProperties

def make_vocabulary(vector_length):
    vec_generator = VectorsWithProperties(vector_length, algebra=HrrAlgebra(), properties = [HrrProperties.UNITARY, HrrProperties.POSITIVE])
    vocab = spa.Vocabulary(vector_length, pointer_gen=vec_generator)
    return vocab

In [ ]:
# @title Figure settings

logging.getLogger('matplotlib.font_manager').disabled = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # perfrom high definition rendering for images and plots
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

In [ ]:
# @title Plotting functions

def plot_similarity_matrix(sim_mat, labels, values = False):
    """
    Plot the similarity matrix between vectors.

    Inputs:
    - sim_mat (numpy.ndarray): similarity matrix between vectors.
    - labels (list of str): list of strings which represent concepts.
    - values (bool): True if we would like to plot values of similarity too.
    """
    with plt.xkcd():
        plt.imshow(sim_mat, cmap='Greys')
        plt.colorbar()
        plt.xticks(np.arange(len(labels)), labels, rotation=45, ha="right", rotation_mode="anchor")
        plt.yticks(np.arange(len(labels)), labels)
        if values:
            for x in range(sim_mat.shape[1]):
                for y in range(sim_mat.shape[0]):
                    plt.text(x, y, f"{sim_mat[y, x]:.2f}", fontsize = 8, ha="center", va="center", color="green")
        plt.title('Similarity between vector-symbols')
        plt.xlabel('Symbols')
        plt.ylabel('Symbols')
        plt.show()

def plot_training_and_choice(losses, sims, ant_names, cons_names, action_names):
    """
    Plot loss progression over training as well as predicted similarities for given rules / correct solutions.

    Inputs:
    - losses (list): list of loss values.
    - sims (list): list of similartiy matrices.
    - ant_names (list): list of antecedance names.
    - cons_names (list): list of consequent names.
    - action_names (list): full list of concepts.
    """
    with plt.xkcd():
        plt.subplot(1, len(ant_names) + 1, 1)
        plt.plot(losses)
        plt.xlabel('Training number')
        plt.ylabel('Loss')
        plt.title('Training Error')
        index = 1
        for ant_name, cons_name, sim in zip(ant_names, cons_names, sims):
            index += 1
            plt.subplot(1, len(ant_names) + 1, index)
            plt.bar(range(len(action_names)), sim.flatten())
            plt.gca().set_xticks(range(len(action_names)))
            plt.gca().set_xticklabels(action_names, rotation=90)
            plt.title(f'{ant_name}, not*{cons_name}')

def plot_choice(sims, ant_names, cons_names, action_names):
    """
    Plot predicted similarities for given rules / correct solutions.
    """
    with plt.xkcd():
        index = 0
        for ant_name, cons_name, sim in zip(ant_names, cons_names, sims):
            index += 1
            plt.subplot(1, len(ant_names) + 1, index)
            plt.bar(range(len(action_names)), sim.flatten())
            plt.gca().set_xticks(range(len(action_names)))
            plt.gca().set_xticklabels(action_names, rotation=90)
            plt.ylabel("Similarity")
            plt.title(f'{ant_name}, not*{cons_name}')

In [ ]:
# @title Set random seed

import random
import numpy as np

def set_seed(seed=None):
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)

set_seed(seed = 42)

In [ ]:
# @title Helper functions

set_seed(42)

symbol_names = ['monarch','heir','male','female']
discrete_space = sspspace.DiscreteSPSpace(symbol_names, ssp_dim=1024, optimize=False)

objs = {n:discrete_space.encode(n) for n in symbol_names}

objs['king'] = objs['monarch'] * objs['male']
objs['queen'] = objs['monarch'] * objs['female']
objs['prince'] = objs['heir'] * objs['male']
objs['princess'] = objs['heir'] * objs['female']

bundle_objs = {n:discrete_space.encode(n) for n in symbol_names}

bundle_objs['king'] = (bundle_objs['monarch'] + bundle_objs['male']).normalize()
bundle_objs['queen'] = (bundle_objs['monarch'] + bundle_objs['female']).normalize()
bundle_objs['prince'] = (bundle_objs['heir'] + bundle_objs['male']).normalize()
bundle_objs['princess'] = (bundle_objs['heir'] + bundle_objs['female']).normalize()

bundle_objs['princess_query'] = (bundle_objs['prince'] - bundle_objs['king']) + bundle_objs['queen']

new_symbol_names = ['dollar', 'peso', 'ottawa', 'mexico-city', 'currency', 'capital']
new_discrete_space = sspspace.DiscreteSPSpace(new_symbol_names, ssp_dim=1024, optimize=False)

new_objs = {n:new_discrete_space.encode(n) for n in new_symbol_names}

cleanup = sspspace.Cleanup(new_objs)

new_objs['canada'] = ((new_objs['currency'] * new_objs['dollar']) + (new_objs['capital'] * new_objs['ottawa'])).normalize()
new_objs['mexico'] = ((new_objs['currency'] * new_objs['peso']) + (new_objs['capital'] * new_objs['mexico-city'])).normalize()

card_states = ['red','blue','odd','even','not','green','prime','implies','ant','relation','cons']
encoder = sspspace.DiscreteSPSpace(card_states, ssp_dim=1024, optimize=False)
vocab = {c:encoder.encode(c) for c in card_states}

for a in ['red','blue','odd','even','green','prime']:
    vocab[f'not*{a}'] = vocab['not'] * vocab[a]

action_names = ['red','blue','odd','even','green','prime','not*red','not*blue','not*odd','not*even','not*green','not*prime']
action_space = np.array([vocab[x] for x in action_names]).squeeze()

rules = [
    (vocab['ant'] * vocab['blue'] + vocab['relation'] * vocab['implies'] + vocab['cons'] * vocab['even']).normalize(),
    (vocab['ant'] * vocab['odd'] + vocab['relation'] * vocab['implies'] + vocab['cons'] * vocab['green']).normalize(),
]

---

# Section 1: Analogies. Part 1

In this section we will construct a simple analogy using Vector Symbolic Algebras. The question we are going to try and solve is "King is to the queen as the prince is to X."

In [ ]:
# @title Video 1: Analogy 1

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display

class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)

def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents

video_ids = [('Youtube', '2tR4fHvL1Jk'), ('Bilibili', 'BV1fS411P7Ez')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_analogy_part_one")

## Coding Exercise 1: Royal Relationships

We're going to start by considering our vocabulary. We will use the basic discrete concepts of monarch, heir, male, and female.

Let's create the objects we know about by combinatorially expanding the space: 

1. King is a male monarch
2. Queen is a female monarch
3. Prince is a male heir
4. Princess is a female heir

Complete the missing parts of the code to obtain correct representations of new concepts.

In [ ]:
set_seed(42)

vector_length = 1024
symbol_names = ['MONARCH', 'HEIR', 'MALE', 'FEMALE']
vocab = make_vocabulary(vector_length)
vocab.populate(';'.join(symbol_names))

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete correct relations for creating new concepts.")

###################################################################
vocab.add('KING', vocab['MONARCH'] * vocab['MALE'])
vocab.add('QUEEN', vocab['MONARCH'] * ...)
vocab.add('PRINCE', vocab['HEIR'] * vocab['MALE'])
vocab.add('PRINCESS', ... * vocab['FEMALE'])

In [ ]:
#to_remove solution

vocab.add('KING', vocab['MONARCH'] * vocab['MALE'])
vocab.add('QUEEN', vocab['MONARCH'] * vocab['FEMALE'])
vocab.add('PRINCE', vocab['HEIR'] * vocab['MALE'])
vocab.add('PRINCESS', vocab['HEIR'] * vocab['FEMALE'])

Now, we can take an explicit approach. We know that the conversion from king to queen is to unbind male and bind female, so let's apply that to our prince object and see what we uncover.

At first, in the cell below, let's recover `queen` from `king` by constructing a new `query` concept, which represents the unbinding of `male` and the binding of `female.`

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete correct relation for creating `query` object to compare with `queen`.")
###################################################################

vocab.add('QUERY_QUEEN', (vocab[...] * ~vocab[...]) * vocab[...])

In [ ]:
#to_remove solution

vocab.add('QUERY_QUEEN', (vocab['KING'] * ~vocab['MALE']) * vocab['FEMALE'])

Then, let's see if this new query object bears any similarity to anything in our vocabulary.

In [ ]:
object_names = list(vocab.keys())
sims = np.zeros((len(object_names), len(object_names)))

for name_idx, name in enumerate(object_names):
    for other_idx in range(name_idx, len(object_names)):
        sims[name_idx, other_idx] = sims[other_idx, name_idx] = spa.dot(vocab[name], vocab[object_names[other_idx]])

plot_similarity_matrix(sims, object_names, values = True)

The above similarity plot shows that applying that operation successfully converts king to queen. Let's apply it to 'prince' and see what happens. Now, `query` should represent the `princess` concept.

In [ ]:
vocab.add('QUERY_PRINCESS', (vocab['PRINCE'] * ~vocab['MALE']) * vocab['FEMALE'])

sims = np.zeros((len(object_names), len(object_names)))

for name_idx, name in enumerate(object_names):
    for other_idx in range(name_idx, len(object_names)):
        sims[name_idx, other_idx] = sims[other_idx, name_idx] = spa.dot(vocab[name], vocab[object_names[other_idx]])

plot_similarity_matrix(sims, object_names, values = True)

Here, we have successfully recovered the princess, completing the analogy.

This approach, however, requires explicit knowledge of the construction of the objects.  Let's see if we can just work with the concepts of 'king,' 'queen,' and 'prince' directly.

In the cell below, construct the `princess` concept using only `king,` `queen`, and `prince.`

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete correct relation for creating `query` object to compare with `princess`.")
###################################################################

vocab.add('QUERY_PRINCESS_2', (vocab[...] * ~vocab[...]) * vocab[...])

In [ ]:
#to_remove solution

# objs['query'] = (objs['prince'] * ~objs['king']) * objs['queen']
vocab.add('QUERY_PRINCESS_2', (vocab['PRINCE'] * ~vocab['KING']) * vocab['QUEEN'])

In [ ]:
object_names = list(vocab.keys())
sims = np.zeros((len(object_names), len(object_names)))

for name_idx, name in enumerate(object_names):
    for other_idx in range(name_idx, len(object_names)):
        sims[name_idx, other_idx] = sims[other_idx, name_idx] = spa.dot(vocab[name], vocab[object_names[other_idx]])

plot_similarity_matrix(sims, object_names, values = True)

Again, we see that we have recovered the princess by using our analogy.

That said, the above depends on knowing that the representations are constructed using binding. Can we do something similar through the bundling operation? Let's try that out.

Reassing concept definitions using bundling operation.

In [ ]:
vocab = vocab.create_subset(['MONARCH','HEIR','FEMALE','MALE'])
vocab.add('KING', (vocab['MONARCH'] + vocab['MALE']).normalized())
vocab.add('QUEEN', (vocab['MONARCH'] + vocab['FEMALE']).normalized())
vocab.add('PRINCE', (vocab['HEIR'] + vocab['MALE']).normalized())
vocab.add('PRINCESS', (vocab['HEIR'] + vocab['FEMALE']).normalized())

But now that we are using an additive model, we need to take a different approach. Instead of unbinding the king and binding the queen, we subtract the king and add the queen to find the princess from the prince.

Complete the code to reflect the updated mechanism.

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete correct relation for creating `query` object to compare with `princess`.")
###################################################################

vocab.add('QUERY_PRINCESS', ((vocab[...] - vocab[...]) + vocab[...]).normalized())

In [ ]:
#to_remove solution

vocab.add('QUERY_PRINCESS', ((vocab['PRINCE'] - vocab['KING']) + vocab['QUEEN']).normalized())

In [ ]:
object_names = list(vocab.keys())
sims = np.zeros((len(object_names), len(object_names)))

for name_idx, name in enumerate(object_names):
    for other_idx in range(name_idx, len(object_names)):
        sims[name_idx, other_idx] = sims[other_idx, name_idx] = spa.dot(vocab[name], vocab[object_names[other_idx]])

plot_similarity_matrix(sims, object_names, values = True)

This is a messier similarity plot due to the fact that the bundled representations interact with all their constituent parts in the vocabulary.  That said, we see that 'princess' is still most similar to the query vector. 

This approach is more like what we would expect from a `word2vec` embedding.

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_royal_relationships")

---

# Section 2: Analogies (Part 2)

Estimated timing to here from start of tutorial: 15 minutes

In this section, we will construct a database of data structures that describe different countries. Materials are adopted from [Kanerva (2010)](https://cdn.aaai.org/ocs/2243/2243-9566-1-PB.pdf).

In [ ]:
# @title Video 2: Analogy 2

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display

class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)

def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents

video_ids = [('Youtube', 'OB3hzhM7Ois'), ('Bilibili', 'BV1TZ421g7G5')]
tab_contents = display_videos(video_ids, W=854, H=480)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_analogy_part_two")

## Coding Exercise 2: Dollar of Mexico

This is going to be a little more involved, because to construct the data structure we are going to need vectors that don't just represent values that we are reasoning about, but also vectors that represent different roles data can play. This is sometimes called a slot-filler representation, or a key-value representation.

At first, let us define concepts and cleanup object.

In [ ]:
set_seed(42)

symbol_names = ['DOLLAR','PESO', 'OTTAWA','MEXICO_CITY','CURRENCY','CAPITAL']
vocab = make_vocabulary(vector_length)
vocab.populate(';'.join(symbol_names))

cleanup = sspspace.Cleanup(vocab)

Now, we will define `Canada` and `Mexico` concepts by integrating the available information together. You will be provided with `Canada` object and your task is to complete for `Mexico` one.

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete `mexico` concept.")
###################################################################

vocab.add('CANADA', (vocab['CURRENCY'] * vocab['DOLLAR'] + vocab['CAPITAL'] * vocab['OTTAWA']).normalized())
vocab.add('MEXICO', (vocab['CURRENCY'] * ... + vocab['CAPITAL'] * ...).normalized())

In [ ]:
#to_remove solution

vocab.add('CANADA', (vocab['CURRENCY'] * vocab['DOLLAR'] + vocab['CAPITAL'] * vocab['OTTAWA']).normalized())
vocab.add('MEXICO', (vocab['CURRENCY'] * vocab['PESO'] + vocab['CAPITAL'] * vocab['MEXICO_CITY']).normalized())

We would like to find out Mexico's currency. Complete the code for constructing a `query` which will help us do that. Note that we are using a cleanup operation.

In [ ]:
###################################################################
## Fill out the following then remove
raise NotImplementedError("Student exercise: complete `query` concept which will be similar to currency in Mexico.")
###################################################################

vocab.add('QUERY_MX_CURRENCY', vocab['MEXICO'] * ~(... * ~...))

In [ ]:
#to_remove solution

vocab.add('QUERY_MX_CURRENCY', vocab['MEXICO'] * ~(vocab['CANADA'] * ~vocab['DOLLAR']))

In [ ]:
object_names = list(vocab.keys())
sims = np.zeros((len(object_names), len(object_names)))

for name_idx, name in enumerate(object_names):
    for other_idx in range(name_idx, len(object_names)):
        sims[name_idx, other_idx] = sims[other_idx, name_idx] = spa.dot(vocab[name], vocab[object_names[other_idx]])

plot_similarity_matrix(sims, object_names)

After cleanup, the query vector is the most similar with the 'peso' object in the vocabularly, correctly answering the question.  

Note, however, that the similarity is not perfectly equal to 1.  This is due to the scale factors applied to the composite vectors 'canada' and 'mexico', to ensure they remain unit vectors, and due to cross talk. Crosstalk is a symptom of the fact that we are binding and unbinding bundles of vector symbols to produce the resultant query vector. The constituent vectors are not perfectly orthogonal (i.e., having a dot product of zero) and as such the terms in the bundle interact when we measure similarity between them.

In [ ]:
# @title Submit your feedback
content_review(f"{feedback_prefix}_dollar_of_mexico")

---
# The Big Picture

*Estimated timing of tutorial: 45 minutes*

In this tutorial, we observed three scenarios where we used the basic operations to solve different analogies and engage in structured learning. Those of you familiar with word embeddings from Natural Language Processing (NLP) might already be familiar with the idea of interpretable semantics on vector representations. This bonus tutorial helps to show how this can be accomplished in a different way. The ability to recreate different phenomena in different paradigms often gives us a great way to compare and contrast model mechanisms and we hope that this bonus tutorial has given you a curiosity to dive a bit deeper and start experimenting further with what you can accomplish using these great open-source tools!